# Langton's Ant

In [2]:
import numpy as np


class CellularAutomaton:
    shape = None
    state = None
    radius = None
    rule = None

    # Define first element of alphabet implicitly as the "unset" cell state.
    # Define second element of alphabet implicitly as the seed cell state.
    alphabet = None
    alphabet_size = None

    def __init__(self, shape, alphabet=[0, 1], radius=1, rule=0, init_state="unset"):
        self.shape = shape
        self.alphabet = alphabet
        self.alphabet_size = len(self.alphabet)
        self.radius = radius
        self.rule = rule
        self.init_state_unset()

        match init_state:
            case "seed":
                self.init_state_seed()
            case "random":
                self.init_state_random()

    def init_state_unset(self):
        self.state = np.full(self.shape, self.alphabet[0])

    def init_state_seed(self):
        center = np.floor(np.asarray(self.shape) / 2).astype(np.int32)
        self.state[tuple(center)] = self.alphabet[1]

    def init_state_random(self):
        self.state = np.random.choice(self.alphabet, self.shape)
        self.state[: self.radius] = self.state[-self.radius :] = 0

    def step_cell(self, state_neighborhood):
        return (self.rule // np.power(self.alphabet_size, state_neighborhood)) % self.alphabet_size

    def step(self):
        # Deviating to the assignment the border size was adapted to be sufficient, i.e. exactly r.
        # The assignment text does not differentiate between r=2 and r=1 case and sets border to 2 fixed.
        state_neighborhood = np.zeros_like(self.state)
        for i in range(-self.radius, self.radius + 1):
            state_neighborhood[self.radius : -self.radius] += self.state[self.radius + i : -self.radius + i or None] * np.power(
                self.alphabet_size, 2 * self.radius - i - self.radius
            )
        self.state = self.step_cell(state_neighborhood)

    def print_state(self):
        state = np.copy(self.state).astype(str)
        mask = self.state == self.alphabet[0]
        state[mask] = "."
        state[~mask] = "#"
        print("".join(state.astype(str)))

    def animate(self, steps):
        self.print_state()
        for _ in range(steps):
            self.step()
            self.print_state()

In [3]:
ca = CellularAutomaton(shape=(84,), alphabet=(0, 1), radius=1, rule=150, init_state="seed")
ca.print_state()

__________________________________________#_________________________________________


In [4]:
ca.animate(50)

__________________________________________#_________________________________________
_________________________________________###________________________________________
________________________________________#_#_#_______________________________________
_______________________________________##_#_##______________________________________
______________________________________#___#___#_____________________________________
_____________________________________###_###_###____________________________________
____________________________________#_#___#___#_#___________________________________
___________________________________##_##_###_##_##__________________________________
__________________________________#_______#_______#_________________________________
_________________________________###_____###_____###________________________________
________________________________#_#_#___#_#_#___#_#_#_______________________________
_______________________________##_#_##_##_#_##_##_#_##___________